In [ ]:
# Celda 2: Importaciones
import requests
import pandas as pd
import sqlite3
import time
from datetime import datetime
import json
from IPython.display import display, clear_output
import ipywidgets as widgets

✅ Módulos importados correctamente


In [ ]:
pip install 

In [1]:
from apify_client import ApifyClient


ModuleNotFoundError: No module named 'apify_client'

In [ ]:
from apify_client import ApifyClient

client = ApifyClient("<YOUR_API_TOKEN>")

# Define the input for the Actor
actor_input = {
    "searchStringsArray": ["ramen"],
    "locationQuery": "New York, USA",
    "maxCrawledPlacesPerSearch": 10,
    "language": "en",
}

# Run an Actor with an input
print("Running the Actor...")
actor_run = apify_client.actor('compass/crawler-google-places').start(run_input=actor_input)

print("🚀 Actor was started")
print("💾 Check your run here: https://console.apify.com/actors/runs/%(id)s" % {"id": actor_run["id"]})